In [7]:
import numpy as np
import pandas as pd
import json
import seaborn as sns

### LOADING THE JSON FILE

In [ ]:
f = open('data.json')
data = json.load(f)
f.close()

In [3]:
type(data)

dict

In [4]:
Cities = list(data.keys())
len(Cities)

623

## Create DataFrame from  Dictionary "data"

In [6]:
Data = pd.DataFrame()                               # Create an Empty Dataframe

for city in Cities:
    try:
        # Convert the city-wise restaurant data to pandas Dataframe
        DF = pd.DataFrame(data[city]['restaurants']).T  
        
        # add City column for each restaurants in that city
        City = [city for i in range(DF.shape[0])]
        DF['City'] = City
        
        # Because no sub-region so considering city as sub-region
        SubRegion = City                                          
        DF['SubRegion'] = SubRegion
        
        # Concatenate to main Dataframe
        Data = pd.concat([Data,DF])
        
    except:
        
        Subregions = list(data[city].keys())
        for region in Subregions:
            
            # Convert the city-wise restaurant data to pandas Dataframe
            DF = pd.DataFrame(data[city][region]['restaurants']).T
            
            # City
            City = [city for i in range(DF.shape[0])]
            # Add City column for each restaurants in that city
            DF['City'] = City
            
            # Sub-region
            SubRegion = [region for i in range(DF.shape[0])]
            
            # add sub-region column for each restaurants in that city
            DF['SubRegion'] = SubRegion
            Data = pd.concat([Data,DF])
        
Data.shape

(181404, 13)

### DICTIONARY TO CSV

In [8]:
### Save our original Dataframe
Data.to_csv('Swiggy_Restaurants.csv')

In [9]:
# Read the saved dataframe
df = pd.read_csv('Swiggy_Restaurants.csv')

In [10]:
menus = []
for i in range(Data.shape[0]):
    try:
        menus.append(', '.join([key for key in Data.menu[i].keys()]))
    except:
        menus.append(np.NaN)
print(len(menus))
print(df.shape[0])

df['menu'] = menus

181404
181404


# CLEANING THE DATA

In [11]:
df = df.rename(columns={'Unnamed: 0':'id', 'name':'Restaurant'})
df.drop_duplicates(subset='id', keep='first', inplace = True)
#df.set_index('id', inplace=True)
df.shape

(160621, 14)

### Rearrange the column order

In [12]:
df = df.loc[:,['id','Restaurant','City','SubRegion','cost','cuisine','rating','rating_count','lic_no',
               'address','menu','link', 'type', 'price']]
df.head()

,id,Restaurant,City,SubRegion,cost,cuisine,rating,rating_count,lic_no,address,menu,link,type,price
0,567335,AB FOODS POINT,Abohar,Abohar,₹ 200,"Beverages,Pizzas",--,Too Few Ratings,22122652000138,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...","Burger, Pasta Must Try, Chiness Appetizer, Fri...",https://www.swiggy.com/restaurants/ab-foods-po...,NaN,NaN
1,531342,Janta Sweet House,Abohar,Abohar,₹ 200,"Sweets,Bakery",4.4,50+ ratings,12117201000112,"Janta Sweet House, Bazar No.9, Circullar Road,...","Breakfast, Drinks (beverages), Combos, Sweets,...",https://www.swiggy.com/restaurants/janta-sweet...,NaN,NaN
2,158203,theka coffee desi,Abohar,Abohar,₹ 100,Beverages,3.8,100+ ratings,22121652000190,"theka coffee desi, sahtiya sadan road city","Recommended, Family Binge Combos, Hot Coffee, ...",https://www.swiggy.com/restaurants/theka-coffe...,NaN,NaN
3,187912,Singh Hut,Abohar,Abohar,₹ 250,"Fast Food,Indian",3.7,20+ ratings,22119652000167,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR","Recommended, Soya Chaap, Gravy Item, Chinese H...",https://www.swiggy.com/restaurants/singh-hut-n...,NaN,NaN
4,543530,GRILL MASTERS,Abohar,Abohar,₹ 250,"Italian-American,Fast Food",--,Too Few Ratings,12122201000053,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...","Veg Burgers, Non-Veg Burgers, Veg Wraps, Non V...",https://www.swiggy.com/restaurants/grill-maste...,NaN,NaN


### CHECKING THE NULL VALUES

In [13]:
Null = df.isnull().sum().reset_index().rename({0:'Null'}, axis = 1)
Null['% Null'] = round((Null['Null']/df.shape[0])*100, 0)
Null.sort_values(by='% Null', ascending = False).style.background_gradient()

,index,Null,% Null
12,type,148591,93.000000
13,price,148590,93.000000
4,cost,12177,8.000000
5,cuisine,12145,8.000000
7,rating_count,12132,8.000000
8,lic_no,12274,8.000000
9,address,12132,8.000000
10,menu,12031,7.000000
0,id,0,0.000000
1,Restaurant,101,0.000000


* `Type & price` columns have more than 90% null values
* Drop `Type & price` columns
* Drop null values row also.

In [14]:
df = df.drop(['type','price'], axis =1)
df.dropna(inplace = True, axis = 0)
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)
df.shape

(148290, 12)

### Cleaning the data
`Noida and Noida-1` are the same city to Provide a single name `Noida`

In [15]:
df.City.replace(['Noida-1','Noida'],'Noida',inplace=True)

# Exploratory Data Analysis

## Rating

In [16]:
Rating = []
for i in range(df.shape[0]):
    try:
        Rating.append(float(df.rating[i]))     
    except:
        #print(Data.rating[i])
        if df.rating[i] == '--' or 'NA':
            Rating.append(np.NaN)                # Where rating is not givem keep it null
        else:
            print(df.rating[i])
print(len(Rating))
print(df.shape[0])
df.rating = Rating
df.rating.dtypes

148290
148290


dtype('float64')

In [29]:
df.cost = df.cost.str.split(' ',1, expand=True)[1].astype('int')
df.cost.dtypes

dtype('int64')

In [36]:
#Q1 - 1.How many cities (including subregions) where Swiggy is having its restaurants listed?

print(df['SubRegion'].nunique())

820


In [37]:
#Q2 - How many cities  (don't include subregions) where Swiggy is having their restaurants listed?

print(df['City'].nunique())

551


In [38]:
#Q3 - The Subregion of Delhi with the maximum number of restaurants listed on Swiggy?
delhi_restaurants = df[df['City'] == 'Delhi']

# Group the DataFrame by 'SubRegion' and count the occurrences
subregion_counts = delhi_restaurants.groupby('SubRegion', as_index=False)['Restaurant'].count()
# Find the subregion with the maximum number of restaurants listed
max_subregion = subregion_counts.loc[subregion_counts['Restaurant'].idxmax()]
print(max_subregion['SubRegion'],max_subregion['Restaurant'])

Indirapuram 1279


In [39]:
# Q4 - Name the top 5 Most Expensive Cities in the Datasets.

expensive_cities = df.groupby('City', as_index=False)['cost'].mean()
expensive_cities['cost'] = expensive_cities['cost'].round(2)
expensive_cities_sorted = expensive_cities.sort_values(by='cost', ascending=False)
top_5_expensive_cities = expensive_cities_sorted.head(5)
top_5_expensive_cities

,City,cost
216,Hinganghat,18962.44
394,North-goa,468.96
183,Gangtok,466.67
362,Mussoorie,460.00
491,South-goa,442.86


In [40]:
# Q5 - List out the top 5 Restaurants with Maximum ratings

# Convert the 'rating' column to numeric data type, ignoring non-numeric and NA values
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Sort the DataFrame in ascending order based on the 'rating' column
top_5_restaurants_min_ratings = df.sort_values(by='rating', ascending=True).head(5)
top_5_restaurants_max_ratings = df.sort_values(by='rating', ascending=False).head(5)
# Print the top 5 restaurants with minimum ratings
print("Top 5 Restaurants with Maximum Ratings:")
print("")
print(top_5_restaurants_max_ratings[['City','Restaurant', 'rating']])
print("")
print("Top 5 Restaurants with Minimum Ratings:")
print("")
print(top_5_restaurants_min_ratings[['City','Restaurant', 'rating']])

Top 5 Restaurants with Maximum Ratings:

            City                                   Restaurant  rating
20647  Bangalore                              Shawarma Indiah     5.0
10687  Bangalore  JUST CREAMERY - Artisanal Healthy Ice Cream     5.0
13058  Bangalore                           Fresh crunch pizza     5.0
51977      Delhi                                  Just Dig In     5.0
59832      Delhi                                  Just Dig In     5.0

Top 5 Restaurants with Minimum Ratings:

              City                        Restaurant  rating
34240   Chandigarh           Ice Cream and Shakes Co     1.0
18539    Bangalore                   Persian Delight     1.1
144921    Varanasi  Champaran Mutton Hundy & Biryani     1.2
51869        Delhi                    THE TARI STORY     1.2
68076      Gurgaon      The Hyderabadi Biryani House     1.2


In [41]:
#Q6 - Top 10 cities as per the number of restaurants listed?
Top = df.City.value_counts(ascending=False).reset_index().rename({'index':'City', 'City':'No of Restaurant '}, axis=1).head(10)
Top

,City,No of Restaurant
0,Bangalore,15288
1,Delhi,14048
2,Pune,12412
3,Hyderabad,9996
4,Chennai,9940
5,Kolkata,8271
6,Mumbai,6064
7,Ahmedabad,3538
8,Chandigarh,3311
9,Lucknow,2842


In [42]:
df.rating_count.unique()

array(['Too Few Ratings', '50+ ratings', '100+ ratings', '20+ ratings',
       '500+ ratings', '1K+ ratings', '5K+ ratings', '10K+ ratings'],
      dtype=object)

In [43]:
review = {}
for rating in df.rating_count.unique():
    if rating == 'Too Few Ratings':
        review[rating] = 0
    else:
        try:
            review[rating] = int(rating.split('+')[0])
        except:
            if rating.split('+')[0][-1] == 'K':
                review[rating] = int(rating.split('+')[0][:-1])*1000
review

{'Too Few Ratings': 0,
 '50+ ratings': 50,
 '100+ ratings': 100,
 '20+ ratings': 20,
 '500+ ratings': 500,
 '1K+ ratings': 1000,
 '5K+ ratings': 5000,
 '10K+ ratings': 10000}

#### Let's add a `Reviews` column in the dataframe with the help `rating_count`.

In [44]:
reviews = []
for i in range(df.shape[0]):
    reviews.append(review[df.rating_count[i]])
print(len(reviews))
print(df.shape[0])

df['Reviews'] = reviews

148290
148290


In [69]:
#Q7 - Name the top 5 Most Popular Restaurants in Pune.
d = df.sort_values(by = 'Reviews', ascending=False)[['Restaurant', 'SubRegion', 'City','rating', 'rating_count','Reviews']]
d[d.City == 'Pune'].head(5)

,Restaurant,SubRegion,City,rating,rating_count,Reviews
122173,Kinara's Maratha Darbar,Kothrud,Pune,4.1,1K+ ratings,1000
132448,Irani Cafe,Magarpatta,Pune,4.3,1K+ ratings,1000
128741,Burger King,PCMC,Pune,4.0,1K+ ratings,1000
129987,McDonald's,Aundh,Pune,4.2,1K+ ratings,1000
132389,Godavari Snacks,Magarpatta,Pune,4.2,1K+ ratings,1000


In [78]:
#Q8 -Which SubRegion in Delhi is having the least expensive restaurant in terms of cost?
delhi_restaurants = df[df['City'] == 'Delhi']
subregion_mean_cost = delhi_restaurants.groupby('SubRegion')['cost'].mean()
least_expensive_subregion = subregion_mean_cost.idxmin()

least_expensive_subregion 

'Sector 18'

In [77]:
#Q9 - Top 5 most popular restaurant chains in India?
popular_restaurant_chains = df.groupby('Restaurant').size().reset_index(name='Count')
popular_restaurant_chains = popular_restaurant_chains.sort_values(by='Count', ascending=False)
top_5_popular_chains = popular_restaurant_chains.head(5)
print("Top 5 Most Popular Restaurant Chains in India:")
top_5_popular_chains[['Restaurant', 'Count']]

Top 5 Most Popular Restaurant Chains in India:


,Restaurant,Count
28539,Domino's Pizza,443
73612,Pizza Hut,319
49040,KFC,309
53384,Kwality Walls Frozen Dessert and Ice Cream Shop,299
11753,Baskin Robbins,276


In [76]:
#Q10 - Which restaurant in Pune has the most number of people visiting?
d = df.sort_values(by = 'Reviews', ascending=False)[['Restaurant', 'SubRegion', 'City','rating', 'rating_count','Reviews']]
d[d.City == 'Pune'].head(1)

,Restaurant,SubRegion,City,rating,rating_count,Reviews
122173,Kinara's Maratha Darbar,Kothrud,Pune,4.1,1K+ ratings,1000


In [95]:
#Q11 - Top 10 Restaurant in Patna w.r.t rating 
df[ df['City']=='Patna' ].sort_values(by='rating',ascending = False).head(10)[['Restaurant','City','rating']]

,Restaurant,City,rating
119104,Kwality Walls Frozen Dessert and Ice Cream Shop,Patna,4.6
119304,Slurpy Shakes,Patna,4.6
119338,Cupcake Bliss Cake & Desserts,Patna,4.5
119361,Burger It Up,Patna,4.5
119292,Kouzina Kafe The Food Court,Patna,4.5
119593,GARAGE KITCHEN,Patna,4.5
119321,The Dessert Zone,Patna,4.4
119370,Pure Veg Meals by Lunchbox,Patna,4.4
119547,KINGDOM OF MOMOS,Patna,4.4
119310,Indiana Burgers,Patna,4.4
